In [ ]:
# !python3 -m spacy download xx_ent_wiki_sm

In [4]:
import pandas as pd
import numpy as np
import re

In [7]:
real = pd.read_excel('data/data.xlsx')
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

real = real[real['shop_name'].isin(target_shops)].copy()
real['shop_name'] = real['shop_name'].apply(lambda x: shops_rename[x])
real = real.drop_duplicates(subset=['name'])
real.sample(2)

,shop_name,name,quantity,price,sum
50369,БИЛЛА,СЫР МОНТЕРЕЙ 45% РАД,0.272,629.3,171.17
50439,БИЛЛА,КЕТЧУП ИТАЛ 350Г,1.000,54.9,54.90


In [8]:
real.head()

,shop_name,name,quantity,price,sum
19,ДИКСИ,МИНТАЙ ФИЛЕ Б/К СВ/МОР П/ПАК 8,1.0,199.90,199.90
20,ДИКСИ,БЗМЖ МОЛОКО СТРАНА ВАСИЛЬКИ У/,1.0,49.99,49.99
21,ДИКСИ,СУШКИ ТАРАЛЛИНИ С ЧЕСНОКОМ 180,1.0,29.99,29.99
22,ДИКСИ,ЯЙЦО КУРИНОЕ СТОЛОВОЕ 1КАТЕГОР,1.0,63.99,63.99
23,ДИКСИ,ПЕЧЕНЬЕ ЮБИЛЕЙНОЕ ВИТАМИНИЗИРО,2.0,29.99,59.98


In [9]:
clean = pd.read_csv('clean_data/all_clean.csv')

FileNotFoundError: [Errno 2] File clean_data/all_clean.csv does not exist: 'clean_data/all_clean.csv'

In [11]:
tmp = pd.read_csv('benchmarks/standard.csv')

In [12]:
def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''

In [13]:
# tmp['Название'].apply(find_weight_or_volume)

In [14]:
tmp['Название'] = tmp['Название'].str.lower()

In [5]:
from product_dict import PRODUCT_DICT

class NormalizeNames:
    def __init__(self, name: pd.Series):
        self.df = pd.DataFrame(name.copy()).rename(columns={'name': 'name_norm'})
        self.df['product_norm'] = None
        self.df['brand_norm'] = None
        
        self.slash_brands = {
            'кп/': 'красная птица',
            'кд/': 'каждый день',
            'lay`s/': 'lays',
            'кр/цена': 'красня цена',
            'д/ваня': 'дядя ваня',
            'дом.в/дер.': 'домик в деревне',
        }

        self.slash_product = {
            'б/йог': 'биойогурт',
            'сух/завтрак': 'сухой завтрак',
            'з/щ': 'зубная щётка',
            'йогурт/смусси': 'йогурт',
            'т/бумага': 'туалетная бумага',
            'укроп/петрушк': 'зелень',
            'б/полотенца': 'бумажные полотенца',
            'з/паста': 'зубная паста',
            'т/мыло': 'мыло',
            'ж/мыло': 'жидкое мыло',
            'бум/салф': 'бумажные полотенца',
            'з/п': 'зубная паста',
            'шамп/бал': 'шампунь',
            'с/порошок': 'порошок',
            'ш/р': 'шариковая ручка',
            'т/бум': 'тулетная бумага',
            
        }
        self.dot_brands = {
            'д.в д': 'домик в деревне',
            'д.в.д': 'домик в деревне',
            'хл.сп': 'хлебный спас',
            'к.ц.': 'красная цена',
            'кр.ц.': 'красная цена',
            'кр.цена': 'красная цена',
            'пр!ст': 'просто',
            'орео': 'oreo',
            'п.св': 'первая свежесть',
            'раст': 'растишка',
            'гранд дуэт': 'grand duet',
            'прост.': 'простоквашино',
            'прост ': 'простоквашино',
            'mil gen': 'miller genuine',
            'шарл': 'шарлиз',
            'домашняя кухн': 'домашняя кухня',
            'крикет': 'cricket',
            'домик в дерев': 'домик в деревне',
            'акт.': 'актвия',
            'озор.пчел': 'озорная пчелка',
            'вор.сух': 'воронцовские сухарики',
            'dr.oetker': 'dr.oetker',
            'б.ю.александров': 'б. ю. александров',
            'mr.ricco': 'mr. ricco',
            'geese': 'geese',
            'dr.korner': 'dr. korner',
            'dr.oetker': 'dr. oetker',
            'alp.gold': 'alpen gold',
            'mil.gen.dr' : 'miller',
            'с.прид': 'сады придонья',
            'g.gold': 'gourmet gold',
            'colg.з/п': 'colgate',
            'с.пудовъ': 'с. пудовъ',
            'сибирск.коллекция': 'сибирская коллекция',
            '(данон)': 'danone',
            'lay`': 'lays',
            'кока-кола': 'coca-cola',
            'марс': 'mars',
            'нестле': 'nestle',
            'брест-литовск': 'брест-литовский',
            "lay's": 'lays',
            'хенкель': 'henkel',
            'gl.vil': 'global village',
            'bar': 'barilla',
            'tol': 'tolli',
            'юбил.': 'юбилейное',
            'астор': 'astoria',
            'бар.': 'барни',
            'рот фронт': 'ротфронт',
            'роллт': 'роллтон',
            'верхов.': 'верховье',
            'мист.': 'мистраль',
            'иммун': 'иммунеле',
            'рест.': 'рестория',
            'зд.мен.': 'здоровое меню',
            ' вв ': 'вкусвилл',
            'зелен. линия': 'зеленая линия',
            'комм.': 'коммунарка',
            'волоконовско': 'волоконовское',
            'олвейс': 'always',
            'палмолив': 'palmolive',
            'жатецкий гусь': 'zatecky gus',
            'советские традиц': 'советские традиции',
            'президент': 'president',
            'стародв': 'стародворские',
            'альметте': 'almette',
            'дом в дер': 'домик в деревне',
            'дом дер': 'домик в деревне',
            'дом.в дер.': 'домик в деревне',
            'виола виттр': 'виола виттрока',
            'золот ларец': 'золотой ларец',
            'клинск': 'клинский',
            'добр ': 'добрый',
            'добр. ': 'добрый',
            'рублев': 'рублевский',
            'кремлев': 'кремлевский',
            'гессер': 'gosser',
            'воронц': 'воронцовские',
            'гранола': 'granola',
            'r.sp.': 'ritter sport',
            'sиб.кол.': 'сибирская коллекция',
            'яр.бройл.': 'ярославский бройлер',
            'кузм.': 'кузмино',
            'виола': 'viola',
            'чупа чупс': 'chupa chups',
            'наше солнышк': 'наше солнышко',
            'ролл': 'роллтон',
            'зева': 'zewa',
            'б.ю.алекс.': 'б.ю.александров',
            'мамба': 'mamba',
            'эком.': 'ecomilk',
            'савушк.': 'савушкин хуторок',
            'милка': 'milka',
            'виолетта': 'Violette',
            'марк.пер.': 'маркет перекресток',
            'национ.': 'националь',
            'пр.кав.': 'предгорье кавказа',
            'велк.': 'велком',
            'байс.': 'baisad',
            'ростагроэкс': 'ростагроэкспорт',
            'слав.': 'славянка',
            'варен.': 'варенька',
            'хейнек.': 'heineken',
            'гринфилдголден': 'greenfield',
            'стр.вас.': 'страна васильки',
            'бархатные ручк': 'бархатные ручки',
            'велле': 'velle',
            'терре аллегре': 'terre allegre',
            'зелен.линия': 'зеленая линия',
            'ильина': 'от ильиной',
            'данон': 'danone',
            'юби.': 'юбилейное',
            'больш.кружка': 'большая кружка',
            'рязан.': 'рязаночка',
            'вдох.': 'вдохновение',
            'озби': 'семейка озби',
            'ландлибе': 'landliebe',
            'асен.фер.': 'асеньевская ферма',
            'приос.': 'приосколье',
            'б.ю. алексан': 'б.ю. александров',
            'б.ю.алексан': 'б.ю. александров',
            'б.ю.александров': 'б.ю. александров',
            'б.ю. александров': 'б.ю. александров',
            'азов.кф': 'азовская кф',
            'феликс': 'felix',
            'з.боч': 'золотая бочка',
            'пр.ява': 'принцесса ява',
            'золот.ярлык': 'золотой ярлык',
            'село зел.': 'село зеленое',
            'оral-в': 'оral-b',
            'рог.': 'рогачевъ',
            'золотая симф.': 'золотая симфония',
            'золот стан': 'золотой стандарт',
            'mентос': 'mentos',
            'men.': 'mentos',
            'дан.': 'danone',
            'петр ': 'петр I',
            'ява ': 'ява',
            'лд ': 'LD',
            'люб.': 'любимый',
            'act.': 'activia',
            'сенатор': 'senator',
            'мальборо': 'marlboro',
            'филип моррис': 'philip morris',
            'филлип моррис': 'philip morris',
            'винстон': 'winston',
            'бонд': 'bond',
            'ротманс': 'rothmans',
            'эссе': 'esse',
            'bon': 'bonduelle',
            'магги': 'maggi',
            'чист.лин.': 'чистая линия',
            'рост.': 'ростагроэкспорт',
            'залесск.фермер': 'залесский фермер',
            'kаmis': 'kamis',
            'мистер пропер': 'mr. proper',
            'лейз': 'lays',
            'natur.': 'natural',
            'alw.': 'always',
            'tur.dies.': 'turbo diesel',
            'мдб': 'мясной дом бородина',
            'три короч.': 'три корочки',
            'pur.': 'purina',
            'nest.': 'nestle',
            'bio баланс': 'bio balance',
            'посид.': 'посиделкино',
            'сл.слоб.': 'сладкая слобода',
            'coca-c.': 'coca-cola',
            'хугарден': 'hoegaarden',
            'черног.': 'черноголовка',
            'мерси': 'mersi',
            'чоко пай': 'chocopie',
            'мос.кар.': 'московский картофель',
            'maxib.': 'maxibon',
            'р.хол.': 'русский холод',
            'extr.': 'extreme',
            'свитл.': 'свитлогорье',
            'дом.кух.': 'домашняя кухня',
            'дом кух': 'домашняя кухня',
            'китекат': 'kitekat',
            'контекс': 'contex',
            'лукоил': 'lukoil',
            'орбит': 'orbit',
            'пепси': 'pepsi',
            'простокваш': 'простоквашино',
            'простоквашино': 'простоквашино',
            'раули мерло': 'rauli merlot',
            'ядрена коп': 'ядрена копоть',
            'ядрена': 'ядрена копоть',
            'тетри': 'tetri',
            'зол.капли': 'золотые капли',
            'шато тамань': 'chateau tamagne',
            'DR.OETK.': 'dr. oetker',
            'баб.': 'бабаевский',
            'меллер': 'meller',
            'кр.лин.': 'красная линия',
            'мос.пр.': 'московский провансаль',
            'м.к.': 'морской котик',
            'обер.': 'обережье',
            'б.бер.': 'балтийский берег',
            'ясно солны': 'ясно солнышко',
            'коров.': 'коровка',
            'любят.': 'любятово',
            'рус.нива': 'русская нива',
            'алекс.': 'б. ю. александров',
            'спрайт': 'sprite',
            'фанта': 'fanta',
            'фр.сад': 'фруктовый сад',
            'дюарс': 'dewars',
            'вологод.пломбир': 'вологодский пломбир',
            'greenf': 'greenfield',
            'hochl': 'hochland',
            'nesc': 'nescafe',
            'ч лин': 'чистая линия',
            'pres': 'president',
            'бабаев': 'бабаевский',
            'дош': 'доширак',
            'бр-лит': 'брест-литовский',
            'lor.': 'lorenz naturals',
            'dan': 'danone',
            'bond': 'bonduelle',
            'colg': 'colgate',
            'сады прид.': 'сады придонья',
            'ч.линия': 'чистая линия',
        }
        self.brands = [
            'тараллини', 'каприз', 'president', 'золотой ларец', 'heineken', 'посиделкино', 'cricket', 'кузмино', 'праздничная',
            'золотой стандарт', 'hochland', 'шишкин лес', 'красная цена', 'bio balance', 'провансаль', 'карусель', 'волковская пивоварня',
            'rich', 'ролтон', 'окей', 'сиббиттер', 'славянка', 'моя цена', 'mars', 'чистая линия', 'озорная пчелка', 'я самая',
            'варвара краса', 'оral-b', 'figaro', 'брест-литовский', 'billa', 'верховье', 'nescafe', 'маракас', 'хлебцы-молодцы',
            'aura family', 'орловский', 'danone', 'рот-фронт', 'bonduelle', 'залесский фермер', 'от ильиной', 'роллтон', 'baisad',
            'агро-альянс', 'стародворские', 'miller genuine', 'рублевский', 'круглый год', 'chupa chups', 'золотой ярлык', 'даниссимо',
            'lays', 'российский', 'мираторг', 'б.ю.александров', 'LD', 'senator', 'рязаночка', 'любодарово', 'море рядом', 'родные просторы',
            'черноголовка', 'geese', 'коммунарка', 'наше солнышко', 'здоровое меню', 'mr. proper', 'maggi', 'индана', 'орион', 'просто',
            'extreme', 'milka', 'хлебный спас', 'felix', 'русский холод', 'мягкий знак', 'свитлогорье', 'доброфлот', 'purina', 'viola',
            'рестория', 'село зеленое', 'winston', 'contex', 'dirol', 'малиновка', 'овощная семейка', 'сибирская коллекция', 'granola',
            'ява', 'maxibon', 'щелковский', 'маркет перекресток', 'пятерочка', 'mersi', 'palmolive', 'nemoloko', 'маруся', 'philip morris',
            'бабаевский', 'барни', 'miller', 'иммунеле', 'esse', 'из углича', 'mccormick', 'henkel', 'heinz', 'велком', 'простоквашино',
            'kitekat', 'топ шеф', 'яков давыдов', 'zatecky gus', 'tolli', 'бифилайф', 'ростагроэкспорт', 'три корочки', 'восточный гость',
            'gourmet gold', 'волоконовское', 'первым делом', 'хлебозавод', 'фрекен бок', 'саф-момент', 'предгорье кавказа', 'кириешки',
            'националь', 'colgate', 'с. пудовъ', 'дикси', 'петр I', 'первак', 'завтрак+', 'мясной дом бородина', 'цукерман', 'тульский',
            'ritter sport', 'растишка', 'мера вкуса', 'landliebe', 'степановна', 'юбилейное', 'dr. korner', 'ля фам', 'виола виттрока',
            'рогачевъ', 'кока кола', 'марьин', 'вкусвилл', 'страна васильки', 'rothmans', 'foxlite', 'бархатные ручки', 'как раньше',
            'astoria', 'юрма', 'белая долина', 'gosser', 'ярославский бройлер', 'савушкин хуторок', 'kamis', 'm&ms', 'marlboro',
            'eco-botanica', 'always', 'pepsi', 'dr. oetker', 'воронцовские сухарики', 'мистраль', 'асеньевская ферма', 'советские традиции',
            'deluxe', 'kotex', 'коптильный двор', 'клинский', 'гиагинский', 'перекресток', 'фрутоняня', 'лента', 'золотая симфония',
            'bond', 'краски лета', 'mr. ricco', 'любимый', 'chocopie', 'кремлевский', 'молти', 'velle', 'фруто-няня', 'особый',
            'воронцовские', 'oreo', 'приосколье', 'nestle', 'вдохновение', 'б.ю. александров', 'activia', 'orbit', 'alpen gold',
            'lorenz naturals', 'отличная цена', 'сады придонья', 'домашняя кухня', 'добрый', 'агуша', 'варенька', 'океан', 'barilla',
            'мясной дом', 'almette', 'greenfield', 'дмитровский', 'красна ягода', 'Violette', 'кузя', 'global village', 'азовская кф',
            'принцесса ява', 'ecomilk', 'сладкая слобода', 'natural', 'крестьянское', 'белевская пастила', 'ашан', 'turbo diesel',
            'московский картофель', 'oral-b', 'ротфронт', 'царский вкус', 'овсяная изюминка', 'доширак', 'большая кружка', 'зеленая линия',
            'донской табак', 'актвия', 'huggies', 'mamba', 'томмолоко', 'coca-cola', 'домик в деревне', 'lukoil', 'шарлиз', 'первая свежесть',
            'му-у', 'grand duet', 'zewa', 'hoegaarden', 'семейка озби', 'terre allegre', 'mentos', 'золотая бочка', 'магнит'
        ]
        
    @staticmethod
    def remove_all_words_which_contains_number(name, product, brand):
        # Todo: Создать словарь с брендами, у которых есть цифры, например, 365 ДНЕЙ, 7 DAYS
        brands_with_numeric = [
            '365 дней', '7 days', '6 соток', '5 морей', '4 сезона', '7up', '3 glocken', '1toy', '48 копеек',
            'j7', 'got2b', 'хлебозавод 28', '4life','4 life' '36 копеек', 'байкал', '101 зерно', '5 океанов', 'j-7',
            '48коп', 'петр I', '7days', '7 злаков', '7злаков'
        ]
        
        for br in brands_with_numeric:
            if br in name:
                brand = br
                if brand == '48коп':
                    brand = '48 копеек'
                name = name.replace(br, '')
                break
        name = ' '.join(re.sub(r'\w*\d\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_all_words_which_contains_slash(self, name, product, brand):
        for key, value in self.slash_brands.items():
            if key in name:
                brand = value
                name = name.replace(key, ' ')
                break
        
        for key, value in self.slash_product.items():
            if key in name:
                product = value
                name = name.replace(key, ' ')
                break
        name = name.replace('/', ' ')
#         name = ' '.join(re.sub(r'\w*/\w*', ' ', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_punctuation_symbols(self, name, product, brand):
        for key, value in self.dot_brands.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        
        tmp = [re.sub(r'([.,!?%:№*\(|\)])', ' ', word).strip() for word in name.split()]
        name = ' '.join(word[:-1] if word.endswith('-') else word for word in tmp)
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_one_and_two_chars(name: str):
#         whitelist = ['с', 'в', 'и', 'из']
        whitelist = ['fa']
        return ' '.join(x.strip() for x in name.split() if len(x) > 2 or x in whitelist)
#         return ' '.join(re.sub(r'\b\w{,2}\b', ' ', word).strip() for word in name.split())
    
    
    def find_brands(self, name, product, brand):
        for br in self.brands:
            if br in name:
                brand = br
                name = name.replace(br, '')
                break
    
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_words_in_blacklist(name: str):
        blacklist = ['бзмж', 'вес', 'для', 'нарезной', 'пшен', 'тенд', 'белый', 'premium', 'декор', 'ранний',
                    'струч', 'короткоплодные', 'венские', 'хлебопек', 'репчатый', 'упак', 'мытая',
                    'дуболистный', 'красный', 'газовая', 'средний', 'слив', 'перс', 'груш', 'классич', 'прод',
                     'раств', 'зерно', 'ябл-бан', 'выгодно', 'красные', 'цветы', 'раст', 'кета', 'косметик',
                    'короткоплодн', 'абр', 'жел', 'шейкер', 'гречн', 'сдоб', 'н-р', '-сл', '[м]', 'вбд',
                     'россия', 'по-домашнему', 'по-корейски', 'киш-миш', 'тд-холдинг', 'дой-пак', 'фрито',
                    'кисло-сладкий', 'рж-пш', 'оквашино', 'нар', 'фас', 'изд', 'охл', 'бел', 'дет', 'паст', 'one',
                    'сливочный', 'зеленью', 'плавл', 'ассорти', 'классическое', 'суп&соус', 'чизбургер', 'ломтик', 'чизбург',
                    'пец-хаас', 'кукурузный', 'ванильный', 'пищевой', 'баварская', 'мульт', 'крупные', 'оливлайн',
                     'черные', 'val', 'плав', 'ломт', 'асс', 'фин', 'избр', 'плавленый', 'финский', 'творожный',
                    'рисунком', 'арт', 'р-р', 'гладкие', 'крас', 'премиум', 'лбп', 'пачка', 'сушеные', 'хлебзернопрод', 'азербайджан', 'розовые',
                    'смо', 'изо', 'ивиа', 'прозр', 'барное', 'светлое', 'темное', 'ультр', 'среднеплодные', 'гладк', 'прополис', 'чесноком', 'карам',
                    'детский', 'яблоко-вишня', 'ишка', 'фасованные', 'детское', 'ржаной', 'украинск', 'оливковый', 'инд', 'окваш', 'моц', 'столовое', 'бокс',
                    'молочный', 'нарез', 'столовая', 'весенние', 'зимой', 'зер', 'зерненый', 'бород', 'белокочанная', 'ранняя', 'домашние', 'морская', 'мелкая', 'рус', 'кривское',
                    'классический', 'ригли', 'вал', 'дрож', 'ябл-черная', 'соев', 'ван', 'москов', 'докторская', 'сух', 'дрожжевое', 
                     'слоеное', 'кокосом', 'зеленый', 'спм', 'вареная', 'выпечки', 'курин', 'пшеничная', 'мытый', 'берканские', 'сладкие',
                    'навелин', 'круглый', 'пинк леди', 'желтый', 'вильямс', 'топл', 'белые', 'спз', 'кдв', 'цыпл-брой', 'сетке',
                     'длинноплодный', 'соленые', 'ский', 'бок', 'тер', 'овские', 'атл', 'маринован', 'гов', 'бородинский', 'рожки', 'пинк', 'хлебоп', 'адриатика',
                    'черн-овес', 'пмм', 'жёлтый', 'дск', 'ветке', 'колотый', 'мга', 'домашний', 'подложка', 'цена', 'детские', 'палермо', 'ромашка', 'фетакса', 'отб',
                    'горький', 'сметанковыймдж', 'комсомольская правда', 'сальчичон', 'гос', 'горшочке', 'красна ягода', 'самокл', 'каймак', 'мягкий', 'кос', 'завар',
                    'шок-ор', 'литовский', 'семейка', 'парижский', 'шашл', 'отборный', 'бакинские', 'отб', 'свежий', 'нарезка', 'форелл', 'стер', 'экон', 'нарезка',
                    'голден', 'дров', 'гладкий', 'больш', 'бриут', 'даниловский', 'франц', 'рислинг', 'тсх', 'тепличные', 'овские', 'стол ']
        return ' '.join(word for word in name.split() if word.strip() not in blacklist)
    
    @staticmethod
    def replace_with_product_dict(name: str):
        return ' '.join(PRODUCT_DICT.get(word.strip(), word) for word in name.split())
    
    @staticmethod
    def replace_most_commen_reductions(name: str):
        reductions = {
            'ср-во': 'средство',
            'филе-кусок': 'филе',
            'нап': 'напиток',
            'крем-мыло': 'мыло',
            'мини-сушки': 'сушки',
            'лук-репка': 'лук',
            'отбел': 'отбеливатель',
            'биотвор': 'биотворог',
            'биойог': 'биойогурт',
            'вин': 'вино',
            'припр': 'приправа',
            'мол': 'молоко',
            'шок': 'шоколад',
            'конф': 'конфеты',
            'жев': 'жевательная резинка',
            'колб': 'колбаса',
            'пакет-майка': 'пакет',
            'твор': 'творог',
            'печен': 'печенье',
            'штруд': 'штрудель',
            'пашт': 'паштет',
            'твор': 'творог',
            'йог': 'йогурт',
            'морож': 'мороженое',
            'салф': 'салфетки',
            'рез': 'резинка',
            'бат': 'батон',
            'майон': 'майонез',
            'марм': 'мармелад',
            'дес': 'десерт',
            'жеврезинка': 'жевательная резинка',
            'шамп': 'шампанское',
            'шокол': 'шоколад',
            'лим кислота': 'лимонная кислота',
            'том.паста': 'томатная паста',
            'ваф': 'вафли',
            'пельм': 'пельмени',
            'спагет': 'спагетти',
            'мюс': 'мюсли',
            'кор': 'коре',
            'наполн': 'наполнитель',
            'пирож': 'пирожное',
            'макар': 'макароны',
            'порош': 'порошок',
            'tоматы': 'томаты',
            'сред': 'средство',
            'свечи-мини': 'свечи',
            'проклад': 'прокладки',
            'хлоп': 'хлопья',
            'шам': 'шампунь',
            'нект': 'нектар',
            'зп': 'зубная паста',
            'сухар': 'сухари',
            'сосис': 'сосиски',
            'приправ': 'приправа',
            'котл': 'котлета',
            'прокл': 'прокладки',
            'салфет': 'салфетки',
            'мармел': 'мармелад',
            'печ': 'печенье',
            'напит': 'напиток',
            'багета': 'багет',
            'горош': 'горошек',
            'мак': 'макароны',
            'семеч': 'семечки',
            'кренд': 'крендели',
            'пасх': 'пасха',
            'зажиг': 'зажигалка',
            'батонч': 'батончик',
            'сырчечилспаг': 'сыр',
            'ароматиз': 'ароматизатор',
            'наб': 'набор',
            'антиперсп': 'антиперспант',
            'бальз-опол': 'бальзам',
            'бзмжсыр': 'сыр',
            'био-йогу': 'биойогурт',
            'био-йогурт': 'биойогурт',
            'биотв': 'биотворог',
            'бумаг': 'бумага',
            'вермиш': 'вермишель',
            'гель-эксп': 'гель',
            'зажигалок': 'зажигалка',
            'кетч': 'кетчуп',
            'кнофеты': 'конфеты',
            'крем-мыл': 'мыло',
            'кэфир': 'кефир',
            'лосьон-тон': 'лосьон',
            'макарон': 'макароны',
            'манг': 'манго',
            'морожен': 'мороженое',
            'пирожн': 'пирожное',
            'подг': 'подгузники',
            'презерв': 'презервативы',
            'сосиск': 'сосиски',
            'тамп': 'тампоны',
            'наб': 'набор',
            'биойог': 'биойогурт',
            'кетч': 'кетчуп',
            'кукур': 'кукуруза',
            'бум': 'бумага',
            'разрыхлит': 'разрыхлитель',
            'красит': 'краситель',
        }
        return ' '.join(reductions.get(word.strip(), word) for word in name.split())
        
    def find_scratch_card(self, name, product, brand):
        if 'скретч-карта' in name:
            return pd.Series(['', 'скретч-карта', 'перекресток'])
        return pd.Series([name, product, brand])
    
    @staticmethod
    def find_product(name, product):
        if not product:
            if len(name.split()) == 1:
                if name == 'мол':
                    return pd.Series(['', 'молоко'])
                product = name
                return pd.Series(['', product])
            return pd.Series([name, None]) 
        return pd.Series([name, product])

    @staticmethod
    def find_and_remove_all_english_words(name, brand):
        eng_brands = re.findall(r'\b([a-z]+)\b', name)
        if eng_brands and not brand:
            result = []
            for word in eng_brands:
                name = name.replace(word, '').replace('  ', ' ').strip()
                result.append(word)

            brand = ' '.join(result)
            if 'epica' in brand:
                brand = 'epica'
            return pd.Series([name, brand])

        # If we have brand, we should delete english words in product name
        name = re.sub(r'\b([a-z]+)\b', '', name).replace('  ', ' ').strip()
        return pd.Series([name, brand])
    
    def normalize(self):
        self.df['name_norm'] = self.df['name_norm'].str.lower()
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_number(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_slash(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_with_product_dict)
        
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_punctuation_symbols(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_brands(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_one_and_two_chars)
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_words_in_blacklist)
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_most_commen_reductions)
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_scratch_card(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_and_remove_all_english_words(x['name_norm'], x['brand_norm']), axis=1
        )
#         self.df[['name_norm', 'product_norm']] = self.df.apply(
#             lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1
#         )
        return self.df

ModuleNotFoundError: No module named 'product_dict'

In [126]:
%%time
n = NormalizeNames(real['name'])
tmp = n.normalize()
# Wall time: 2min 18s

CPU times: user 2min 13s, sys: 1.44 s, total: 2min 15s
Wall time: 2min 18s


In [127]:
df = pd.concat([real, tmp], axis=1)[['shop_name', 'name', 'name_norm', 'product_norm', 'brand_norm']]

In [128]:
df.brand_norm.dropna().shape # 13020 -> 13719 -> 13726

(13726,)

In [129]:
df.product_norm.dropna().shape # 350 -> 350

(350,)

In [24]:
from typing import Optional, NoReturn, List, Union
from itertools import combinations
from pymystem3 import Mystem


class Finder:
    """
    Searche and recognize the name, category and brand of a product
    from its description.
    Search is carried out in the collected datasets: `rus_brands.csv`,
    `products.csv`, `all_clean.csv`.
    
    Parameters
    ----------
    data_to_parse: Union[pd.DataFrame, str],
        Data in which product information will be recognized.
        If data is a pd.DataFrane, it must contain the following columns:
        ['name_norm', 'product_norm', 'brand_norm'].
        If the data is a string, it should be normalized:
        service characters removed, lowercase letters, etc.
    path_to_brands : str
        Path to the dataset where brand information is collected.
    path_to_products : str
        Path to the dataset, which collects information about
        the product name and their categories.
    path_to_product_db : str
        The path to the dataset, which collects scrapied
        information about products from chain stores.
        
    Attributes
    ----------
    mystem : class
        A Python wrapper of the Yandex Mystem 3.1 morphological
        analyzer (http://api.yandex.ru/mystem).
        See aslo `https://github.com/nlpub/pymystem3`.
    rus_brands : np.ndarray
        List of Russian brands.
    products : pd.DataFrame
        DataFrame of product names and categories.
    df : pd.DataFrame
        The copy of `data_to_parse`.

    Examples
    --------
    TODO
    """ # TODO: Add Examples в доку
    
    def __init__(self, data_to_parse: Union[pd.DataFrame, str], path_to_brands: str = 'clean_data/rus_brands.csv',
                 path_to_products: str = 'clean_data/products.csv', path_to_product_db='clean_data/all_clean.csv'):
        self.mystem = Mystem()
        
        # DataFrames:
        self.rus_brands = pd.read_csv(path_to_brands)['Бренд'].values
        self.products = pd.read_csv(path_to_products)
        self.product_db = pd.read_csv(path_to_product_db)
        
        columns = ['name_norm', 'product_norm', 'brand_norm']
        if isinstance(data_to_parse, pd.DataFrame):
            self.data = df_to_parse[columns].copy()
        else:
            self.data = pd.DataFrame([[data_to_parse, None, None]], columns=columns)
        self.data['cat_norm'] = None  # Add new column
        
    def find_brands(self, name: str, brand: Optional[str] = None) -> pd.Series:
        """
        Find Russian brands using the dataset `rus_brands.csv`.
        For more accurate recognition, a combination of words in a
        different order is used.

        Parameters
        ----------
        name : str
            Product name.
        brand : str, optional (default=None)
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([name, brand])
        """

        if name and not brand:
            names = set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split())
            for rus_brand in self.rus_brands:
                if rus_brand in names:
                    name = name.replace(rus_brand, '').replace('  ', ' ').strip()
                    return pd.Series([name, rus_brand])
        return pd.Series([name, brand])

    @staticmethod
    def _remove_duplicate_word(arr: List[str]) -> List[str]:
        """
        Remove duplicates in words when one name is a  continuation
        of another: ['вода', 'вода питьевая'] --> ['вода питьевая'].
        
        Parameters
        ----------
        arr : List[str]
            List description of products in different variants.
        
        Returns
        -------
        arr : List[str]
            List description of products without duplicates.
        """

        if max([len(x.split()) for x in arr]) > 1:
            arr = sorted(arr, key=lambda x: len(x.split()))
            one_words = []
            for product in arr.copy():
                if len(product.split()) == 1:
                    one_words.append(product)
                else:
                    for word in one_words:
                        if word in product:
                            arr.remove(word)
        return arr

    def find_product(self, name: str, product: str, category: Optional[str]=None) -> pd.Series:
        """
        Find products name using the dataset `products.csv`.
        For more accurate recognition, a combination of words in a
        different order is used.

        Parameters
        ----------
        name : str
            Product name.
        product : str
            Product description.
        category : str, optional (default=None)
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([name, product, category]) 
        """

        if name and not product:
            names = pd.DataFrame(set([f'{comb[0]} {comb[1]}' for comb in combinations(name.split(), 2)] + name.split()), columns=['Продукт'])
            merge = self.products.merge(names)
            if len(merge):
                product = ', '.join(self._remove_duplicate_word(merge['Продукт'].values))
                category = merge['Категория'].value_counts().index[0]
        return pd.Series([name, product, category])
    
    @staticmethod
    def remove_dash(name: str, product: str) -> str:
        """
        Remove dash from product description.

        Parameters
        ----------
        name : str
            Product name.
        product : str
            Product description.

        Returns
        -------
        str
           Product name without dashs.
        """

        if name and not product:
            name = name.replace('-', ' ')
        return name
    
    def use_mystem(self, name: str, product: str) -> str:
        """
        Use Yandex pymystem3 library to lemmatize words in product descriptions.
        I tried to use pymorphy, but the recognition quality got worse.

        Parameters
        ----------
        name : str
            Product name.
        product : str
            Product description.
            
        Returns
        -------
        str
            Product description after lemmatization.
        """

        if name and not product:
            name = ''.join(self.mystem.lemmatize(name)[:-1])
        return name
    
    def find_category(self, product: str, category: str) -> pd.Series:
        """
        Find a product category using the dataset `products.csv`.

        Parameters
        ----------
        product : str
            Product description.
        category : str
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([product, category]) 
        """
        if product and not cat:
            tmp = self.products[self.products['Продукт'] == product]
            if len(tmp):
                category = tmp['Категория'].values[0]

        return pd.Series([product, category])


    def find_product_by_brand(self, product: str, brand: str, category: str) -> pd.Series:
        """
        If we were able to recognize the product brand,
        but could not recongize the product name,
        we can assign the most common product name for this brand.

        Parameters
        ----------
        product : str
            Product description.
        brand : str
            Product brand.
        category : str
            Product category.

        Returns
        -------
        pd.Series
           pd.Series([product, brand, category]) 
        """

        if brand and not product:
            single_brand_goods = self.product_db[self.product_db['Бренд'] == brand]
            if len(single_brand_goods):
                product = single_brand_goods['Продукт'].value_counts().index[0]
                category = single_brand_goods['Категория'].value_counts().index[0]

        return pd.Series([product, brand, category])
    
    
    def print_logs(self) -> NoReturn:
        """
        Print the number of recognized brands,
        categories and names of goods.
        """
        
        _len = len(data)
        print(
            "Recognized brands: "
            f"{len(self.data['brand_norm'].dropna())}/{_len}, "
            f"products: {len(self.data['product_norm'].dropna())}/{_len}, "
            f"categories: {len(self.data['cat_norm'].dropna())}/{_len}",
            '-'*60, sep='\n', end='\n\n'
        )
    
    def find_all(self):
        
        print('Before:')
        self.print_logs()
        
        # Find brand:
        print('Find brands...')
        self.data[['name_norm', 'brand_norm']] = self.data.apply(lambda x: self.find_brands(x['name_norm'], x['brand_norm']), axis=1)
        self.print_logs()
        
        # Find product and category:
        print('Find product...')
        self.data[['name_norm', 'product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1)
        self.print_logs()
        
        # Remove `-` and the second attempt to find a product:
        print('Remove `-`...')
        self.data['name_norm'] = self.data.apply(lambda x: self.remove_dash(x['name_norm'], x['product_norm']), axis=1)
        self.data[['name_norm', 'product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product(x['name_norm'], x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Use Mystem for lemmatization and the third attempt to find a product:
        print('Lemmatization...')
        self.data['name_norm'] = self.data.apply(lambda x: self.use_mystem(x['name_norm'], x['product_norm']), axis=1)
        self.data[['name_norm', 'product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product(x['name_norm'], x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Find category:
        print('Find category...')
        self.data[['product_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_category(x['product_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        
        # Find product by brand:
        print('Find product by brand...')
        self.data[['product_norm', 'brand_norm', 'cat_norm']] = self.data.apply(lambda x: self.find_product_by_brand(x['product_norm'], x['brand_norm'], x['cat_norm']), axis=1)
        self.print_logs()
        return self.data

In [109]:
%%time
finder = Finder(df)
res = finder.find_all()
res = pd.concat([df['name'], res], axis=1)
# Recognized brands: 17178, products: 30343, categories: 30343

# Recognized brands: 17358, products: 30499, categories: 30499 --> split by '/'

# Recognized brands: 17366, products: 30497, categories: 30497

# Recognized brands: 17528, products: 30495, categories: 30495 # Add new brands

# Wall time: 3min 19s

Before:
Recognized brands: 13719, products: 350, categories: 0
------------------------------------------------------------

Find brands...
Recognized brands: 17528, products: 350, categories: 0
------------------------------------------------------------

Find product...
Recognized brands: 17528, products: 29459, categories: 29109
------------------------------------------------------------

Remove `-`...
Recognized brands: 17528, products: 29558, categories: 29208
------------------------------------------------------------

Lemmatization...
Recognized brands: 17528, products: 29924, categories: 29574
------------------------------------------------------------

Find category...
Recognized brands: 17528, products: 29924, categories: 29924
------------------------------------------------------------

Find product by brand...
Recognized brands: 17528, products: 30495, categories: 30495
------------------------------------------------------------

CPU times: user 3min 14s, sys: 438 ms, 

### TODO:

1. Вынести все собранные словари в отдельные классы
2. Подумать насчёт того, чтобы совместить мой список со списком брендов
3. Переводить всё в .py файлы
4. Настроить красивые логи


In [24]:
q = pd.read_csv('clean_data/products.csv')

In [40]:
q[q['Продукт'] == 'дыня']

,Продукт,Категория


In [151]:
q.drop(index=461).to_csv('clean_data/products.csv', index=False)

In [112]:
q.loc[446]['Категория'] = 'Овощи, фрукты, ягоды'

In [137]:
q.loc[950]

Продукт        изд "мозаика-синтез"
Категория    Товары для мам и детей
Name: 950, dtype: object

In [115]:
q.to_csv('clean_data/products.csv', index=False)

In [284]:
'макароны' in q['Продукт'].values

True

In [60]:
l = ['сгущенка', 'открытка', 'поджарка', 'белок', 'плитка', 'окорочка', 'гвоздика', 'горошек', 'редька', 'рассол', 'пасха', 'кислота лимонная']

In [38]:
q['Категория'].unique()

array(['Красота, гигиена, бытовая химия', 'Хлеб, сладости, снеки',
       'Молоко, сыр, яйца', 'Соусы, орехи, консервы',
       'Макароны, крупы, специи', 'Другое', 'Воды, соки, напитки',
       'Рыба, икра', 'Чай, кофе, сахар', 'Замороженные продукты',
       'Птица, мясо, деликатесы', 'Посуда', 'Овощи, фрукты, ягоды',
       'Товары для мам и детей', 'Товары для дома и дачи',
       'Подборки и готовые блюда', 'Зоотовары', 'Бытовая техника',
       'Алкоголь', 'Дача и гриль'], dtype=object)

In [78]:
tmp = pd.DataFrame.from_dict(white_list, orient='index').reset_index().rename(columns={'index': 'Продукт', 0: 'Категория'})

In [125]:
l = ['яблоко', 'долма', 'разрыхлитель', 'бумага', 'кукуруза']

In [21]:
for i in df.product_norm.dropna().drop_duplicates().values:
    if i not in q['Продукт'].values:
        print(i)

тулетная бумага
шариковая ручка
пучок


In [22]:
tmp = pd.DataFrame(
    [
        ['тулетная бумага', 'Красота, гигиена, бытовая химия'],
        ['шариковая ручка', 'Другое'],
        ['пучок', 'Овощи, фрукты, ягоды'],
    ], columns=['Продукт', 'Категория']
)

In [23]:
pd.concat([q, tmp]).to_csv('clean_data/products.csv', index=False)

In [85]:
q.shape

(1265, 2)

In [88]:
q.drop_duplicates(subset='Продукт').to_csv('clean_data/products.csv', index=False)

In [279]:
from collections import Counter

C = Counter()
def lol(name):
    global C
    for i in name.split():
        C[i]+=1



In [280]:
res[res['product_norm'].apply(lambda x: not bool(x))]['name_norm'].apply(lol);